<a href="https://colab.research.google.com/github/hpzhang94/289g-project/blob/main/BiLSTM_TextClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [72]:
import pandas
import torch
import torch.nn as nn
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
from torch.utils.data import DataLoader, Dataset
import torch.optim as optim
from torchtext.data import get_tokenizer
from torchtext.vocab import GloVe
import numpy as np
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns

from google.colab import drive
drive.mount('/content/drive')

tokenizer = get_tokenizer("basic_english")

device = torch.device('cuda:0')

def getR8():
  r8train = pandas.read_csv('/content/drive/MyDrive/Colab Files/R8-lstm/train.txt', sep='\t', names=['label', 'sentence'])
  r8test = pandas.read_csv('/content/drive/MyDrive/Colab Files/R8-lstm/test.txt', sep='\t', names=['label', 'sentence'])
  return r8train, r8test

def getMovieReview():
  trainX = pandas.read_csv('/content/drive/MyDrive/Colab Files/MovieReview/text_train.txt', sep='\t', names=['sentence'])
  trainY = pandas.read_csv('/content/drive/MyDrive/Colab Files/MovieReview/label_train.txt', sep='\t', names=['label'])
  train = pandas.concat([trainX, trainY], axis=1)
  testX = pandas.read_csv('/content/drive/MyDrive/Colab Files/MovieReview/text_test.txt', sep='\t', names=['sentence'])
  testY = pandas.read_csv('/content/drive/MyDrive/Colab Files/MovieReview/label_test.txt', sep='\t', names=['label'])
  test = pandas.concat([testX, testY], axis=1)
  return train, test

r8train, r8test = getR8()
mr_train, mr_test = getMovieReview()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [70]:
def get_len_stats(df):
  df['length'] = df['sentence'].str.split().str.len()
  print(df['length'].describe())
get_len_stats(mr_train)

count    7108.000000
mean       21.000563
std         9.396583
min         1.000000
25%        14.000000
50%        20.000000
75%        27.000000
max        59.000000
Name: length, dtype: float64


In [75]:
def get_vocab(df):
  counts = 0
  vocab = set()
  for index, row in df.iterrows():
    vocab.update(tokenizer(row['sentence']))
  return vocab

def load_glove_vectors():
  return GloVe(name='840B', dim=300)

def get_embedding_matrix(pretrained, vocab, emb_size=300):
  """ Creates embedding matrix from word vectors"""
  vocab_size = len(vocab) + 2
  vocab_to_idx = {}
  new_vocab = ["", "UNK"]
  W = np.zeros((vocab_size, emb_size), dtype="float32")
  W[0] = np.zeros(emb_size, dtype='float32') # adding a vector for padding
  W[1] = np.random.uniform(-0.25, 0.25, emb_size) # adding a vector for unknown words 
  vocab_to_idx["UNK"] = 1
  i = 2
  for word in vocab:
    W[i] = pretrained.get_vecs_by_tokens([word], lower_case_backup=True)[0]
    vocab_to_idx[word] = i
    new_vocab.append(word)
    i += 1   
  return W, np.array(new_vocab), vocab_to_idx

def encode_sentence(text, vocab_to_idx):
  tokens = tokenizer(text)
  actual_encoding = np.array([vocab_to_idx.get(word, vocab_to_idx["UNK"]) for word in tokens])
  return actual_encoding

def df_mapping(df, vocab_to_idx, label_to_idx=None):
  ## convert sentence to list of indices
  df['encoding'] = df['sentence'].apply(lambda x: encode_sentence(x, vocab_to_idx))
  if label_to_idx:
    df['Y'] = df['label'].map(label_to_idx)
  else:
    df['Y'] = df['label']
  return df

def get_label_mapping(df):
  labels = df['label'].unique()
  label_to_idx = {labels[i]:i for i in range(len(labels))}
  idx_to_label = {i:labels[i] for i in range(len(labels))}
  return label_to_idx, idx_to_label

# pretrained = load_glove_vectors()

# for R8
# vocab = get_vocab(r8train)
# print(f'Vocab size is {len(vocab)}')
# W, vocab2, vocab_to_idx = get_embedding_matrix(pretrained, vocab)
# label_to_idx, idx_to_label = get_label_mapping(r8train)
# train_df = df_mapping(r8train, vocab_to_idx, label_to_idx)
# test_df = df_mapping(r8test, vocab_to_idx, label_to_idx)

# for MR
vocab = get_vocab(mr_train)
print(f'Vocab size is {len(vocab)}')
W, vocab2, vocab_to_idx = get_embedding_matrix(pretrained, vocab)
label_to_idx, idx_to_label = get_label_mapping(mr_train)
train_df = df_mapping(mr_train, vocab_to_idx)
test_df = df_mapping(mr_test, vocab_to_idx)

Vocab size is 16615


In [74]:
def collate(batch, pad_index=0):
    label_list, seq_list, len_list = [], [], []
    for (_seq, _label, _len) in batch:
      label_list.append(_label)
      seq_list.append(torch.tensor(_seq))
      len_list.append(_len)
    return pad_sequence(seq_list, batch_first=True, padding_value=pad_index), torch.tensor(label_list), torch.tensor(len_list)

class MyDataset(Dataset):
  def __init__(self, X, Y):
    self.X = X
    self.y = Y
  
  def __len__(self):
    return len(self.y)
  
  def __getitem__(self, idx):
    return torch.from_numpy(self.X[idx].astype(np.int32)), self.y[idx], len(self.X[idx])

trainX = list(train_df['encoding'])
trainY = list(train_df['Y'])
testX = list(test_df['encoding'])
testY = list(test_df['Y'])
# from sklearn.model_selection import train_test_split
# X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2)
train_ds = MyDataset(trainX, trainY)
train_dl = DataLoader(train_ds, batch_size=64, collate_fn=collate, shuffle=True)
test_ds = MyDataset(testX, testY)
test_dl = DataLoader(test_ds, batch_size=64, collate_fn=collate, shuffle=False)

In [76]:
class LSTM(nn.Module):
  def __init__(self, vocab_size, pretrained, label_size, dimension=128):
    super(LSTM, self).__init__()
    self.embedding = nn.Embedding(vocab_size, 300, padding_idx=0)
    self.embedding.weight.data.copy_(torch.from_numpy(pretrained))
    self.embedding.weight.requires_grd = False ## freeze weights
    self.dimension = dimension
    self.lstm = nn.LSTM(input_size=300, hidden_size=dimension, num_layers=1, batch_first=True, bidirectional=True)
    self.drop = nn.Dropout(p=0.2)
    self.fc = nn.Linear(2*dimension, label_size)
  
  def forward(self, text, text_len):
    text_emb = self.embedding(text)
    packed_input = pack_padded_sequence(text_emb, text_len.cpu(), batch_first=True, enforce_sorted=False)
    packed_output, _ = self.lstm(packed_input)
    output, _ = pad_packed_sequence(packed_output, batch_first=True)

    out_forward = output[range(len(output)), text_len - 1, :self.dimension]
    out_reverse = output[:, 0, self.dimension:]
    out_reduced = torch.cat((out_forward, out_reverse), 1)
    text_fea = self.drop(out_reduced)

    text_fea = self.fc(text_fea)
    text_fea = torch.squeeze(text_fea, 1)
    text_out = torch.sigmoid(text_fea)
    return text_out
  
model = LSTM(len(vocab2), W, len(label_to_idx)).to(device)

In [7]:
# Save and Load Functions

def save_checkpoint(save_path, model, optimizer, valid_loss):

    if save_path == None:
        return
    
    state_dict = {'model_state_dict': model.state_dict(),
                  'optimizer_state_dict': optimizer.state_dict(),
                  'valid_loss': valid_loss}
    
    torch.save(state_dict, save_path)
    print(f'Model saved to ==> {save_path}')


def load_checkpoint(load_path, model, optimizer):

    if load_path==None:
        return
    
    state_dict = torch.load(load_path, map_location=device)
    print(f'Model loaded from <== {load_path}')
    
    model.load_state_dict(state_dict['model_state_dict'])
    optimizer.load_state_dict(state_dict['optimizer_state_dict'])
    
    return state_dict['valid_loss']


def save_metrics(save_path, train_loss_list, valid_loss_list, global_steps_list):

    if save_path == None:
        return
    
    state_dict = {'train_loss_list': train_loss_list,
                  'valid_loss_list': valid_loss_list,
                  'global_steps_list': global_steps_list}
    
    torch.save(state_dict, save_path)
    print(f'Model saved to ==> {save_path}')


def load_metrics(load_path):

    if load_path==None:
        return
    
    state_dict = torch.load(load_path, map_location=device)
    print(f'Model loaded from <== {load_path}')
    
    return state_dict['train_loss_list'], state_dict['valid_loss_list'], state_dict['global_steps_list']


In [79]:
# Training Function

def train(model,
          optimizer,
          train_loader,
          valid_loader,
          criterion = nn.CrossEntropyLoss(),
          num_epochs = 5,
          eval_every = len(train_dl) // 2,
          file_path = '/content/drive/MyDrive/Colab Files/MovieReview',
          best_valid_loss = float("Inf")):
    # print(valid_loader)
    # initialize running values
    running_loss = 0.0
    valid_running_loss = 0.0
    global_step = 0
    train_loss_list = []
    valid_loss_list = []
    global_steps_list = []

    # training loop
    model.train()
    for epoch in range(num_epochs):
        for x, y, length in train_loader:   
            x = x.long().to(device)
            y = y.long().to(device)
            length = length.long().to(device)
            output = model(x, length)

            loss = criterion(output, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # update running values
            running_loss += loss.item()
            global_step += 1

            # evaluation step
            if global_step % eval_every == 0:
                model.eval()
                with torch.no_grad():                    
                  # validation loop
                  for test_x, test_y, test_len in valid_loader:
                      test_x = test_x.long().to(device)
                      test_y = test_y.long().to(device)
                      test_len = test_len.long().to(device)
                      test_output = model(test_x, test_len)

                      loss = criterion(test_output, test_y)
                      valid_running_loss += loss.item()

                # evaluation
                average_train_loss = running_loss / eval_every
                average_valid_loss = valid_running_loss / len(valid_loader)
                train_loss_list.append(average_train_loss)
                valid_loss_list.append(average_valid_loss)
                global_steps_list.append(global_step)

                # resetting running values
                running_loss = 0.0                
                valid_running_loss = 0.0
                model.train()

                # print progress
                print('Epoch [{}/{}], Step [{}/{}], Train Loss: {:.4f}, Valid Loss: {:.4f}'
                      .format(epoch+1, num_epochs, global_step, num_epochs*len(train_loader),
                              average_train_loss, average_valid_loss))
                
                # checkpoint
                if best_valid_loss > average_valid_loss:
                    best_valid_loss = average_valid_loss
                    save_checkpoint(file_path + '/model.pt', model, optimizer, best_valid_loss)
                    save_metrics(file_path + '/metrics.pt', train_loss_list, valid_loss_list, global_steps_list)
    
    save_metrics(file_path + '/metrics.pt', train_loss_list, valid_loss_list, global_steps_list)
    print('Finished Training!')


optimizer = optim.Adam(model.parameters(), lr=0.001)
train(model, optimizer, train_dl, test_dl, num_epochs=30)
y_pred = evaluate(model, test_dl)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """


Epoch [1/30], Step [56/3360], Train Loss: 0.3203, Valid Loss: 0.5554
Model saved to ==> /content/drive/MyDrive/Colab Files/MovieReview/model.pt
Model saved to ==> /content/drive/MyDrive/Colab Files/MovieReview/metrics.pt
Epoch [1/30], Step [112/3360], Train Loss: 0.3208, Valid Loss: 0.5444
Model saved to ==> /content/drive/MyDrive/Colab Files/MovieReview/model.pt
Model saved to ==> /content/drive/MyDrive/Colab Files/MovieReview/metrics.pt
Epoch [2/30], Step [168/3360], Train Loss: 0.3225, Valid Loss: 0.5624
Epoch [2/30], Step [224/3360], Train Loss: 0.3209, Valid Loss: 0.5542
Epoch [3/30], Step [280/3360], Train Loss: 0.3230, Valid Loss: 0.5513
Epoch [3/30], Step [336/3360], Train Loss: 0.3207, Valid Loss: 0.5438
Model saved to ==> /content/drive/MyDrive/Colab Files/MovieReview/model.pt
Model saved to ==> /content/drive/MyDrive/Colab Files/MovieReview/metrics.pt
Epoch [4/30], Step [392/3360], Train Loss: 0.3224, Valid Loss: 0.5615
Epoch [4/30], Step [448/3360], Train Loss: 0.3211, Vali

In [80]:
def evaluate(model, test_loader):
    y_pred = []
    y_true = []

    model.eval()
    with torch.no_grad():
      for test_x, test_y, test_len in test_loader:
        test_x = test_x.long().to(device)
        test_y = test_y.long().to(device)
        test_len = test_len.long().to(device)
        test_output = model(test_x, test_len)
        output = torch.argmax(test_output, dim=1)
        y_pred.extend(output.tolist())
        y_true.extend(test_y.tolist())
    
    accuracy = accuracy_score(y_true, y_pred)
    print(f'Accuracy is {accuracy}')
    return y_pred
    
    
best_model = LSTM(len(vocab2), W, len(label_to_idx)).to(device)
# optimizer = optim.Adam(best_model.parameters(), lr=0.001)

load_checkpoint('/content/drive/MyDrive/Colab Files/MovieReview/model.pt', best_model, optimizer)
pred = pandas.DataFrame({'pred': evaluate(best_model, test_dl)})
pred.to_csv('/content/drive/MyDrive/Colab Files/MovieReview/mr-bilstm-pred.csv')

Model loaded from <== /content/drive/MyDrive/Colab Files/MovieReview/model.pt


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """


Accuracy is 0.7664603263927968
